In [19]:
import pandas as pd
import numpy as np


In [14]:
#1: set up facebook people network, as symmetric matrix with weights all 1

number_of_people = 4039
people_network = np.empty([4039,4039])

fb_connections = np.loadtxt('facebook_combined.txt', delimiter = " ")
for i in fb_connections:
    people_network[int(i[0])][int(i[1])] = 1
    people_network[int(i[1])][int(i[0])] = 1

print people_network

[[ 0.  1.  1. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [28]:
#2: Split people into Republican, Democrat, or Independent
#40.9% call themselves liberal, 28.4% moderate, and 30.7% call themselves conservative 

parties = list(range(number_of_people))
np.random.shuffle(parties)

democrats = parties[:int(len(parties)*0.41)]
republicans = parties[int(len(parties)*0.41)+1:int(len(parties)*0.72)]
independents = parties[int(len(parties)*0.72)+1:]

In [51]:
#3: Add in news sites and fake news sites
num_news_sites = 7
fake_news_sites = 8

news_sites_indices = []
total_network = people_network
for i in range(num_news_sites):
    news_sites_indices.append(number_of_people+i)
    total_network = np.insert(total_network,-1,0,axis = 1)
    total_network = np.insert(total_network,-1,0,axis = 0)

news_sites_indices_political = np.random.choice(3,num_news_sites,replace = False)
    
fake_news_sites_indices = []
for i in range(fake_news_sites):
    fake_news_sites_indices.append(news_sites_indices[-1]+1+i)
    total_network = np.insert(total_network,-1,0,axis = 1)
    total_network = np.insert(total_network,-1,0,axis = 0)

fake_news_sites_indices_political = np.random.choice(2,fake_news_sites,replace = False)

#fake_news_sites_indices holds the indices for the fake news sites
#news_sites_indices holds the indices for the regular news sites
#0 is democratic, 1 is republican, 2 is independent 

In [63]:
#4: Construct weight (W) matrix
weight_for_significant_others = 0.2
weight_given_to_news_sources = 0.1
number_news_connections = 4

for i in range(number_of_people):
    weight_given = 0
    connections = np.nonzero(total_network[i,:] != 0)[0]
    signif_others = [connections[i] for i in np.random.choice(len(connections),2,replace = False)]
    for j in signif_others:
        total_network[i,j] = weight_for_significant_others
        weight_given += weight_for_significant_others
    
    if i in democrats:
        my_news = [news_sites_indices[i] for i in range(len(news_sites_indices)) if news_sites_indices_political[i] == 0]
        my_fake_news = [fake_news_sites_indices[i] for i in range(len(fake_news_sites_indices)) if fake_news_sites_indices_political[i] == 0]
        all_news = np.append(my_fake_news,my_news)
        news_connections = [all_news[i] for i in np.random.choice(len(all_news),number_news_connections,replace = False)]
        print news_connections
    elif i in republicans:
        my_news = [news_sites_indices[i] for i in range(len(news_sites_indices)) if news_sites_indices_political[i] == 1]
        my_fake_news = [fake_news_sites_indices[i] for i in range(len(fake_news_sites_indices)) if fake_news_sites_indices_political[i] == 1]
        all_news = np.append(my_fake_news,my_news)
        news_connections = [all_news[i] for i in np.random.choice(len(all_news),number_news_connections,replace = False)]
        print news_connections
    elif i in independents:
        all_news = np.append(news_sites_indices,fake_news_sites_indices)
        news_connections = [all_news[i] for i in np.random.choice(len(all_news),number_news_connections, replace = False)]
        print news_connections
    
    for a in news_connections:
        total_network[i,int(a)] = weight_given_to_news_sources
        weight_given += weight_given_to_news_sources
        
    weight_left = 1-weight_given
    np.random.shuffle(connections)
    for c in connections:
        if weight_left > 0:
            w = np.random.random()*weight_left
            total_network[i,c] = w
            weight_left = weight_left - w
    
    break
    

[4047.0, 4049.0, 4048.0, 4050.0]


In [36]:
a = np.array([1,2,3])
np.nonzero(a != 1)

(array([1, 2]),)